Part 1: Generate the Daily Return Matrix(DataFrame) for each SP500 Index Constituent Stock in the Study Period

In [220]:
import pandas as pd
import numpy as np
import pickle

Note: In COMPUSTAT database, TWENTY-FIRST CENTURY FOX INC once had two stocks with CUSIP equal to 90130A309 and 90130A408 respectively in SP500 Index. However, in CRSP database, there is only 1 return series for this company's stock with CUSIP equal to 90130A10 (In contrast, there can be a company that once had more than one stock with different CUSIPs in SP500 Index, but in CRSP database, there also exists the corresponding number of return series for this company's stock). Thus, to avoid bug in later analysis, I manually delete the record of stock with CUSIP equal to 90130A408 in the following file.

In [77]:
SP500_component=pd.read_table("./SP500_constitents/SP500 Index Constituents from Compustat.txt",
                              parse_dates=["from","thru"])
SP500_component.head()

,gvkey,iid,gvkeyx,from,thru,conm,indextype,tic,spii,spmi,indexcat,co_conm,co_tic,co_cusip,co_cik,co_sic,co_naics
0,1078,1,3,1964-03-31,NaT,S&P 500 Comp-Ltd,LGCAP,I0003,NaN,10,S&P,ABBOTT LABORATORIES,ABT,002824100,1800.0,2834.0,325412.0
1,1300,1,3,1964-03-31,NaT,S&P 500 Comp-Ltd,LGCAP,I0003,NaN,10,S&P,HONEYWELL INTERNATIONAL INC,HON,438516106,773840.0,9997.0,336413.0
2,1440,1,3,1964-03-31,NaT,S&P 500 Comp-Ltd,LGCAP,I0003,NaN,10,S&P,AMERICAN ELECTRIC POWER CO,AEP,025537101,4904.0,4911.0,2211.0
3,2285,1,3,1964-03-31,NaT,S&P 500 Comp-Ltd,LGCAP,I0003,NaN,10,S&P,BOEING CO,BA,097023105,12927.0,3721.0,336411.0
4,2403,1,3,1964-03-31,NaT,S&P 500 Comp-Ltd,LGCAP,I0003,NaN,10,S&P,BRISTOL-MYERS SQUIBB CO,BMY,110122108,14272.0,2834.0,325412.0


In [78]:
# 有些股票多次经过进入和移出SP500指数,所以这里用unique来处理
company_cusip=pd.Series(SP500_component["co_cusip"].map(lambda x: str(x)[:-1]).unique())
company_cusip.to_csv("./SP500_constitents/SP500 Index Constituents' CUSIP.txt",index=False)

I manually download the 

In [79]:
SP500_return=pd.read_table("./SP500_constitents/SP500 Index Stock Return from CRSP.txt",dtype={"CUSIP":str})
SP500_return.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PERMNO,date,TICKER,COMNAM,TRDSTAT,SECSTAT,PERMCO,CUSIP,RET,BID,ASK
0,10057,19900102,AMT,ACME CLEVELAND CORP,A,R,20020,00462610,0.011494,NaN,NaN
1,10057,19900103,AMT,ACME CLEVELAND CORP,A,R,20020,00462610,0,NaN,NaN
2,10057,19900104,AMT,ACME CLEVELAND CORP,A,R,20020,00462610,0.034091,NaN,NaN
3,10057,19900105,AMT,ACME CLEVELAND CORP,A,R,20020,00462610,0.010989,NaN,NaN
4,10057,19900108,AMT,ACME CLEVELAND CORP,A,R,20020,00462610,0,NaN,NaN


In [80]:
cusip_unique=list(SP500_return["CUSIP"].unique())
not_retrieved=[]

for cusip in SP500_component["co_cusip"]:
    if str(cusip)[:-1] in cusip_unique:
        not_retrieved.append(False)
    else:
        not_retrieved.append(True)

In [81]:
no_data=SP500_component[["from","thru","co_conm","co_tic","co_cusip"]][not_retrieved]
len(no_data)

52

In [82]:
no_data[no_data["from"]>=pd.Timestamp("2019-01-01")]  # 4 becausue list date later than 2018-12-31

,from,thru,co_conm,co_tic,co_cusip
1222,2019-02-08,2019-02-10,COVETRUS INC,CVET,22304C100
1225,2019-03-19,NaT,FOX CORP,FOXA,35137L105
1226,2019-03-19,NaT,FOX CORP,FOXA,35137L204
1227,2019-04-02,NaT,DOW INC,DOW,260557103


In [83]:
no_data[no_data["thru"]<pd.Timestamp("1990-01-01")] # 1 becausue delist date earlier than 1990-01-01

,from,thru,co_conm,co_tic,co_cusip
220,1971-12-31,1989-12-20,RAMADA INC,RAM.3,751328105


In [84]:
no_data[pd.isna(no_data["co_cusip"])] # 2 because no cusip number

,from,thru,co_conm,co_tic,co_cusip
1134,2016-05-13,2016-05-13,NaN,NaN,NaN
1145,2016-09-07,2016-09-07,NaN,NaN,NaN


In [85]:
no_data

,from,thru,co_conm,co_tic,co_cusip
60,1964-03-31,1991-05-06,USX CORP-CONSOLIDATED,MROX.CM,90399Y931
65,1964-03-31,1993-06-30,PITTSTON CO-CONSOLIDATED,PZSM.CM,72599Y937
66,1964-03-31,1993-07-31,RALSTON PURINA-CONSOLIDATED,RACM.CM,75199Z931
74,1964-03-31,1994-09-26,AVATEX CORP,AVATQ,05349F402
76,1964-03-31,1994-12-20,HEXION INC,0141A,428300107
86,1964-03-31,1997-08-05,TXU GAS CO,TXU2,873170005
101,1964-03-31,2000-01-30,CONSOLIDATED NATURAL GAS CO,D2,209615004
115,1964-03-31,2000-11-01,COLUMBIA ENERGY GROUP,NI2,197648009
138,1964-03-31,2003-03-13,AMERICAN AIRLINES GROUP INC,AAL,001765106
158,1964-03-31,2009-06-02,GENERAL MOTORS CO,GM,62010A105


I manually 

In [86]:
mapping=pd.read_table("./SP500_constitents/Mapping of CUSIP between Compustat and CRSP.txt",sep=",")
mapping["cusip_in_crsp"].to_csv("./SP500_constitents/SP500 Index Constituents' CUSIP-Supplement.txt",index=False)

I manually 

In [87]:
SP500_return_s=pd.read_table("./SP500_constitents/SP500 Index Stock Return from CRSP-Supplement.txt",
                             dtype={"CUSIP":str})

In [88]:
SP500_return_c=pd.concat([SP500_return,SP500_return_s],axis=0)
SP500_return_c.index=pd.Index(np.arange(len(SP500_return_c)))
SP500_return_c.head()

,PERMNO,date,TICKER,COMNAM,TRDSTAT,SECSTAT,PERMCO,CUSIP,RET,BID,ASK
0,10057,19900102,AMT,ACME CLEVELAND CORP,A,R,20020,00462610,0.011494,NaN,NaN
1,10057,19900103,AMT,ACME CLEVELAND CORP,A,R,20020,00462610,0,NaN,NaN
2,10057,19900104,AMT,ACME CLEVELAND CORP,A,R,20020,00462610,0.034091,NaN,NaN
3,10057,19900105,AMT,ACME CLEVELAND CORP,A,R,20020,00462610,0.010989,NaN,NaN
4,10057,19900108,AMT,ACME CLEVELAND CORP,A,R,20020,00462610,0,NaN,NaN


In [114]:
company_cusip.name="cusip_in_crsp" 
# company_cusip=pd.Series(SP500_component["co_cusip"].map(lambda x: str(x)[:-1]).unique())
company_cusip_2=pd.Series(SP500_component["co_cusip"].unique())
company_cusip_2.name="cusip_in_compustat"
mapping_2=pd.concat([company_cusip,company_cusip_2],axis=1)
mapping_3=pd.concat([mapping_2,mapping],axis=0)
mapping_3.set_index("cusip_in_crsp",inplace=True)
mapping_3=mapping_3["cusip_in_compustat"] # transform from DataFrame to Series

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [116]:
%time SP500_return_c["CUSIP"]=SP500_return_c["CUSIP"].map(lambda x: mapping_3.loc[x])

CPU times: user 5min 46s, sys: 4.64 s, total: 5min 50s
Wall time: 5min 47s


In [117]:
SP500_return_c["RET"]=(SP500_return_c["RET"].replace(["B","C"],np.nan)).astype(np.float64)

In [123]:
%time daily_return=pd.crosstab(index=SP500_return_c["date"],columns=SP500_return_c["CUSIP"],\
                               values=SP500_return_c["RET"],aggfunc=lambda x:x)

CPU times: user 32.3 s, sys: 1.16 s, total: 33.5 s
Wall time: 33.7 s


In [219]:
daily_return.head()

CUSIP,000886309,00101J106,001055102,001204106,00130H105,001547108,001957505,00206R102,00209A106,002824100,...,G97822103,G98294104,H1467J104,H27178104,H2906T109,H84989104,H8817H100,N53745100,N59465109,V7780T103
date,,,,,,,,,,,,,,,,,,,,,
19900102,0.013605,NaN,-0.020833,-0.032922,NaN,NaN,0.021978,0.013699,NaN,0.018382,...,NaN,NaN,NaN,0.060241,NaN,NaN,NaN,NaN,-0.020619,NaN
19900103,-0.006711,NaN,-0.021277,0.000000,NaN,NaN,-0.008065,-0.017375,NaN,0.003610,...,NaN,NaN,NaN,-0.011364,NaN,NaN,NaN,NaN,-0.036842,NaN
19900104,-0.020270,NaN,-0.021739,-0.017021,NaN,NaN,-0.016260,-0.025147,NaN,-0.001799,...,NaN,NaN,NaN,-0.022989,NaN,NaN,NaN,NaN,-0.010929,NaN
19900105,0.006897,NaN,0.000000,0.012987,NaN,NaN,-0.016529,-0.034623,NaN,-0.010811,...,NaN,NaN,NaN,-0.011765,NaN,NaN,NaN,NaN,0.000000,NaN
19900108,-0.013699,NaN,0.014815,0.012821,NaN,NaN,0.025210,0.016878,NaN,0.005100,...,NaN,NaN,NaN,0.005952,NaN,NaN,NaN,NaN,0.011050,NaN


In [223]:
pickle.dump(daily_return,open("./daily_return","wb"))

Part 2: Generate the SP500 Index Constituent Stock Binary Matrix (DataFrame) for Every Month End in the Study Period

In [124]:
time_index=pd.date_range("1989-12","2019-01",freq="M") # from 1989-12-31 to 2018-12-31

In [129]:
list_time=pd.concat([SP500_component["from"]]*len(time_index),axis=1).T
list_time.head()

,0,1,2,3,4,5,6,7,8,9,...,1218,1219,1220,1221,1222,1223,1224,1225,1226,1227
from,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,...,2018-12-03,2018-12-24,2019-01-02,2019-01-18,2019-02-08,2019-02-15,2019-02-27,2019-03-19,2019-03-19,2019-04-02
from,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,...,2018-12-03,2018-12-24,2019-01-02,2019-01-18,2019-02-08,2019-02-15,2019-02-27,2019-03-19,2019-03-19,2019-04-02
from,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,...,2018-12-03,2018-12-24,2019-01-02,2019-01-18,2019-02-08,2019-02-15,2019-02-27,2019-03-19,2019-03-19,2019-04-02
from,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,...,2018-12-03,2018-12-24,2019-01-02,2019-01-18,2019-02-08,2019-02-15,2019-02-27,2019-03-19,2019-03-19,2019-04-02
from,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,1964-03-31,...,2018-12-03,2018-12-24,2019-01-02,2019-01-18,2019-02-08,2019-02-15,2019-02-27,2019-03-19,2019-03-19,2019-04-02


In [131]:
delist_time=pd.concat([SP500_component["thru"]]*len(time_index),axis=1).T
delist_time=delist_time.replace(delist_time.iloc[0,0],pd.Timestamp('2020-01-01 00:00:00'))
delist_time.head()

,0,1,2,3,4,5,6,7,8,9,...,1218,1219,1220,1221,1222,1223,1224,1225,1226,1227
thru,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,...,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2019-02-10,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01
thru,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,...,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2019-02-10,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01
thru,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,...,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2019-02-10,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01
thru,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,...,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2019-02-10,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01
thru,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01,...,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2019-02-10,2020-01-01,2020-01-01,2020-01-01,2020-01-01,2020-01-01


In [133]:
current_time=pd.concat([pd.Series(time_index)]*len(SP500_component),axis=1)
current_time.head()

,0,1,2,3,4,5,6,7,8,9,...,1218,1219,1220,1221,1222,1223,1224,1225,1226,1227
0,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31,...,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31,1989-12-31
1,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31,...,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31,1990-01-31
2,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28,...,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28,1990-02-28
3,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31,...,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31,1990-03-31
4,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30,...,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30,1990-04-30


In [213]:
stock_consti=pd.DataFrame(((list_time.values<=current_time.values)&(delist_time.values>current_time.values)).
                          astype(np.float),index=time_index,columns=SP500_component["co_cusip"])
stock_consti.head()

cusip_in_compustat,002824100,438516106,025537101,097023105,110122108,134429109,149123101,166764100,191216100,194162103,...,25278X109,150870103,33616C100,879369106,22304C100,049560105,929740108,35137L105,35137L204,260557103
1989-12-31,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-31,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-02-28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-03-31,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-04-30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [214]:
# 剔除重复值
column_frequence=stock_consti.columns.value_counts() # 不会考虑np.nan的情况
duplicate_column=column_frequence[column_frequence>1]
for column_n in duplicate_column.index:
    for i in range(len(stock_consti[column_n].columns)):
        if i==0:
            new_column=stock_consti[column_n].iloc[:,[i]]
        else:
            new_column=new_column+stock_consti[column_n].iloc[:,[i]]
    stock_consti.drop(labels=column_n,axis=1,inplace=True)
    stock_consti=pd.concat([stock_consti,new_column],axis=1)

In [211]:
# stock_consti=stock_consti.T[(1-pd.isna(stock_consti.columns)).astype(bool)].T

In [217]:
stock_consti=stock_consti[daily_return.columns]

In [218]:
stock_consti.head()

cusip_in_compustat,000886309,00101J106,001055102,001204106,00130H105,001547108,001957505,00206R102,00209A106,002824100,...,G97822103,G98294104,H1467J104,H27178104,H2906T109,H84989104,H8817H100,N53745100,N59465109,V7780T103
1989-12-31,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-31,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-02-28,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-03-31,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-04-30,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [222]:
pickle.dump(stock_consti,open("./stock_consti","wb"))